In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

def mean_std_df(df, group_columns, columns_to_agg):
    xdf = df.groupby(group_columns).agg({column : [np.mean, np.std] for column in columns_to_agg})
    xdf.columns = xdf.columns.map("_".join)
    return xdf.reset_index()


In [12]:
exp_seqs = [1, 3, 5]
df = pd.DataFrame()
for exp_seq in exp_seqs:
    df_ = pd.read_csv(f'result/exp_{exp_seq}_client_u_shaped_split_learning/result.csv', header=None)
    # append to df
    df = df.append(df_, ignore_index=True)
df.columns = ['exp_seq', 'epoch', 'total_epochs', 'learning_rate', 'architecture_choice', 'batch_size', 'len_train_dataset', 'len_val_dataset', 'len_test_dataset', 'train_loss', 'train_dia_acc', 'train_sps_acc', 'val_loss', 'val_dia_acc', 'val_sps_acc', 'val_mean_acc', 'is_best_val', 'test_loss', 'test_dia_acc', 'test_sps_acc', 'test_mean_acc', 'time_server_load', 'time_client_training', 'time_server_training', 'time_validation', 'time_test', 'time_communication_server_to_client', 'time_communication_client_to_server', 'time_total', 'size_server_msg', 'size_server_output', 'size_client_head_gradient', 'size_server_model', 'size_client_msg', 'size_client_head_output', 'size_server_gradient']

Training time of client and server

In [15]:
df_mean = mean_std_df(df, ['epoch'], ['time_client_training', 'time_server_training'])
df_mean


fig = go.Figure()

fig.add_trace(go.Scatter(x=df_mean['epoch'], 
                         y=df_mean['time_client_training_mean'],
                         error_y=dict(type='data', array=df_mean['time_client_training_std']), 
                         name='Client Training'))

fig.add_trace(go.Scatter(x=df_mean['epoch'],
                        y=df_mean['time_server_training_mean'],
                        error_y=dict(type='data', array=df_mean['time_server_training_std']), 
                        name='Server Training'))

fig.update_layout(
    title="Time per Epoch",
    xaxis_title="Epoch",
    yaxis_title="Time (s)",
    font=dict(
        # family="Courier New, monospace",
        size=18,
        # color="RebeccaPurple"
    ),
    width=800, 
    height=400
)

fig.show()


In [25]:
def plot(df, 
         x_axis='epoch', 
         y_axiss = ['train_loss', 'val_loss', 'test_loss'], 
         x_axis_title = 'Epoch',
         y_axis_title = 'Loss',
         legend_names=['Train Loss', 'Validation Loss', 'Test Loss'],
         title='Loss per Epoch'):
    
    assert len(y_axiss) == len(legend_names)
    df_mean = mean_std_df(df, [x_axis], y_axiss)

    fig = go.Figure()
    for i, y_axis in enumerate(y_axiss):
        fig.add_trace(go.Scatter(x=df_mean[x_axis],
                                y=df_mean[f'{y_axis}_mean'],
                                error_y=dict(type='data', array=df_mean[f'{y_axis}_std']), 
                                name=legend_names[i]))
    fig.update_layout(
        title=title,
        xaxis_title=x_axis_title,
        yaxis_title=y_axis_title,
        font=dict(
            # family="Courier New, monospace",
            size=18,
            # color="RebeccaPurple"
        ),
        width=800, 
        height=400,
        hovermode="x"
    )
    return fig



In [26]:
fig = plot(df, 
         x_axis='epoch', 
         y_axiss = ['train_loss', 'val_loss', 'test_loss'], 
         x_axis_title = 'Epoch',
         y_axis_title = 'Loss',
         legend_names=['Train Loss', 'Validation Loss', 'Test Loss'],
         title='Loss per Epoch')
fig.show()

In [27]:
fig = plot(df,
            x_axis='epoch', 
            y_axiss = ['train_dia_acc', 'val_dia_acc', 'test_dia_acc'], 
            x_axis_title = 'Epoch',
            y_axis_title = 'Diagnosis Accuracy',
            legend_names=['Train Accuracy', 'Validation Accuracy', 'Test Accuracy'],
            title='Accuracy per Epoch')
fig.show()

In [29]:
fig = plot(df,
            x_axis='epoch', 
            y_axiss = ['train_sps_acc', 'val_sps_acc', 'test_sps_acc'], 
            x_axis_title = 'Epoch',
            y_axis_title = '7 Pt. Accuracy',
            legend_names=['Train Accuracy', 'Validation Accuracy', 'Test Accuracy'],
            title='Accuracy per Epoch')
fig.show()

In [32]:
fig = plot(df,
        x_axis='epoch',
        y_axiss = ['time_total',
                   'time_server_load',
                   'time_client_training',
                   'time_server_training',
                   'time_validation',
                   'time_test',
                   'time_communication_server_to_client', 
                   'time_communication_client_to_server'],
        x_axis_title = 'Epoch',
        y_axis_title = 'Time (s)',
        legend_names=['Total Time',
                        'Server Load',
                        'Client Training',
                        'Server Training',
                        'Validation',
                        'Test',
                        'Server to Client',
                        'Client to Server'],
        title='Time per Epoch')
fig.show()